<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 4: Создайте свой первый алгоритм обучения с глубоким подкреплением с помощью PyTorch: Reinforce. И проверьте его надежность 💪

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/thumbnail.png" alt="thumbnail" width="50%"/>

В этом блокноте вы с нуля создадите свой первый алгоритм обучения с глубоким подкреплением: Reinforcement (также называемый градиентом политики Монте-Карло).

Усиление (Reinforce) - это *метод, основанный на политике*: алгоритм глубокого обучения с подкреплением, который пытается **оптимизировать политику напрямую, не используя функцию значения действия**.

Точнее, Усиление (Reinforce) - это *Метод градиента политики*, подкласс *методов, основанных на политике*, который направлен **на оптимизацию политики непосредственно путем оценки весов оптимальной политики с использованием градиентного подъема**.

Чтобы проверить его надежность, мы собираемся обучить его в двух разных простых средах:
- Cartpole-v1 (Столб тележки-v1)
- PixelcopterEnv (Пиксельный вертолет)

⬇️ Вот пример того, чего **вы достигнете в конце этом блокноте.** ⬇️

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/envs.gif" alt="Environments" width="50%"/>


### 🎮 Окружающая среда: 

- [CartPole-v1](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)
- [PixelCopter](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)

### 📚 RL-Библиотека: 

- Python
- PyTorch


 [GitHub Repo](https://github.com/huggingface/deep-rl-class/issues).

## Цели этого блокнота 🏆
В конце вы будете:
- Уметь **с нуля закодировать алгоритм усиления с использованием PyTorch.**
- Иметь возможность **протестировать надежность вашего агента, используя простые среды.**
- Иметь возможность **отправить своего обученного агента в центр** с хорошим видеоповтором и оценочным баллом 🔥 .

## Эта тетрадь взята из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="50%"/>

На этом бесплатном курсе вы:

- 📖 Изучите глубокое обучение с подкреплением в **теории и на практике**.
- 🧑 💻 Научитесь **использовать известные библиотеки Deep RL**, такие как Stable Baselines3, RL Baselines3 Zoo, CleanRL и Sample Factory 2.0.
- 🤖 Обучите **агентов в уникальных условиях**

## Предпосылки 🏗️
Прежде чем погрузиться в блокнот, вам нужно:

🔲 📚 [Изучите градиенты политики, прочитав Unit 4](https://huggingface.co/deep-rl-course/unit4/introduction)

# Давайте запрограммируем алгоритм усиления с нуля 🔥


Чтобы пройти этот практический подход для сертификации, вам необходимо отправить свои обученные модели в Центр.

- Получите результат >= 350 для `Cart pole-v1`.
- Получите результат >= 5 для `Pixel Copter`.

Чтобы узнать свой результат, перейдите в таблицу лидеров и найдите свою модель, **the result = mean_reward - std_reward**. **Если вы не видите свою модель в таблице лидеров, перейдите в нижнюю часть страницы таблицы лидеров и нажмите на кнопку обновить**.

Для получения дополнительной информации о процессе сертификации ознакомьтесь с этим разделом 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

# Совет 💡
Лучше запустить этот colab в копии на вашем Google Диске, чтобы **в случае истечения времени ожидания** у вас все еще была сохраненная записная книжка на вашем Google Диске, и вам не нужно было заполнять все с нуля.

Для этого вы можете либо нажать `Ctrl + S` либо `File > Save a copy in Google Drive.`

## Запустить на GPU 💪
- Чтобы **ускорить обучение агента, мы будем использовать графический процессор GPU. Перейдите в `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1" width="50%">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2" width="50%">

## Создание виртуального дисплея #

Во время записи нам нужно будет сгенерировать видео-повтор. Для этого с помощью colab **нам нужен виртуальный экран, чтобы иметь возможность отображать среду** (и, таким образом, записывать кадры).

Следовательно, в следующей ячейке будут установлены библиотеки, а также создан и запущен виртуальный экран 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Установка зависимостей #
Первым шагом является установка зависимостей:

- `gym`
- `gym-games`: Дополнительные условия в тренажерном зале, созданные с помощью Pygame.
- `huggingface_hub`: 🤗 работает как центральное место, где любой желающий может делиться моделями и наборами данных и изучать их. В нем есть управление версиями, метрики, визуализации и другие функции, которые позволят вам легко сотрудничать с другими.

Вы можете увидеть здесь все доступные модели усилителей 👉 https://huggingface.co/models?other=reinforce

И вы можете найти все модели обучения с глубоким подкреплением здесь 👉 https://huggingface.co/models?pipeline_tag=reinforcement-learning

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

## Импортируйте пакеты 📦
В дополнение к импорту установленных библиотек, мы также импортируем:

- `imageio`: Библиотека, которая поможет нам сгенерировать повтор видео


In [ ]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym
import gym_pygame

# Hugging Face Hub
from huggingface_hub import notebook_login # Войти в учетную запись Hugging Face, чтобы иметь возможность загружать модели в Хаб.
import imageio

## Проверьте, есть ли у нас графический процессор

Если у вас есть GPU, вы должны увидеть `device:cuda0`

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

Теперь мы готовы реализовать наш алгоритм усиления 🔥

# Первый агент: Играем в Картпол-v1 🤖

## Создайте среду CartPole и поймите, как это работает
### [Окружающая среда 🎮 ](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)


### Почему мы используем такую простую среду, как CartPole-v1?
Как описано в [Советах и рекомендациях по обучению с подкреплением](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html),когда вы внедряете свой агент с нуля, вам нужно **убедиться, что он работает правильно, и найти ошибки в простых средах, прежде чем углубляться**. Поскольку находить ошибки будет намного проще в простых средах.


> Попробуйте подать какой-нибудь “признак жизни” в отношении проблем с игрушками


> Проверьте реализацию, заставив ее запускаться во все более сложных средах (вы можете сравнить результаты с RL zoo). Обычно для этого шага необходимо выполнить оптимизацию по гиперпараметру.
___
### Среда CartPole-v1

> Шест прикреплен с помощью шарнира без привода к тележке, которая движется по рельсам без трения. Маятник устанавливается вертикально на тележку, и цель состоит в том, чтобы уравновесить шест, прикладывая силы в левом и правом направлениях к тележке.



Итак, мы начнем с CartPole-v1. Цель состоит в том, чтобы толкать тележку влево или вправо **так, чтобы шест оставался в равновесии.**

Эпизод заканчивается, если:
- Угол наклона шеста больше, чем ±12°
- Положение тележки больше, чем ±2.4
- Продолжительность эпизода больше, чем 500

Мы получаем вознаграждение 💰 в размере +1 каждый раз, когда полюс остается в равновесии.

In [ ]:
env_id = "CartPole-v1"
# Создание env
env = gym.make(env_id)

# Создание среды оценки
eval_env = gym.make(env_id)

# Получение пространства состояний и пространства действий
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Получение случайного наблюдения

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Получение случайного действия

## Давайте создадим Reinforce архитектуру
Эта реализация основана на двух реализациях:
- [Официальный пример обучения с подкреплением PyTorch](https://github.com/pytorch/examples/blob/main/reinforcement_learning/reinforce.py)
- [Udacity Reinforce](https://github.com/udacity/deep-reinforcement-learning/blob/master/reinforce/REINFORCE.ipynb)
- [Улучшение интеграции с помощью Chris1nexus](https://github.com/huggingface/deep-rl-class/pull/95)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/reinforce.png" alt="Reinforce" width="50%"/>

Итак, мы хотим:
- Два полностью соединенных слоя (fc1 и fc2).
- Использование ReLU в качестве функции активации fc1
- Использование Softmax для вывода распределения вероятностей по действиям

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # Создайте два полностью соединенных (fc-fully connected ) слоя

    def forward(self, x):
        # Определите прямой проход
        # состояние переходит в fc1, затем мы применяем функцию активации ReLU
        # выходы fc1 передаются в fc2
        # Мы выводим softmax
    
    def act(self, state):
        """
        Учитывая состояние, определите действие
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m)
        return action.item(), m.log_prob(action)

### Solution

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m)
        return action.item(), m.log_prob(action)

Я совершаю ошибку, можете ли вы догадаться, где?

- Чтобы выяснить это, давайте сделаем пас вперед:

In [ ]:
debug_policy = Policy(s_size, a_size, 64).to(device)
debug_policy.act(env.reset())

- Здесь мы видим, что ошибка гласит `ValueError: аргумент value для log_prob должен быть тензором`

- Это означает, что `action` в `m.log_prob(action)` должно быть, тензором **, но это не так.**

- Ты знаешь почему? Проверьте функцию act и попытайтесь понять, почему она не работает.

Совет 💡 : Что-то не так в этой реализации. Помните, что **мы хотим выбрать действие из распределения вероятностей по действиям**.


### Правильное решение

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

Используя CartPole, отлаживать было проще, поскольку **мы знаем, что ошибка возникает из-за нашей интеграции, а не из-за нашей простой среды**.

- Поскольку **мы хотим выбрать действие из распределения вероятностей по действиям**, мы не можем использовать `action = np.argmax(m)`, поскольку оно всегда будет выводить действие с наибольшей вероятностью.

- Нам нужно заменить на `action = m.sample()`, который будет отбирать действие из распределения вероятностей P(.|s)

### Давайте построим алгоритм усиленного обучения
Это псевдокод алгоритма усиления:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/pg_pseudocode.png" alt="Policy gradient pseudocode"  width="70%"/>

- Когда мы вычисляем доходность Gt (строка 6), мы видим, что мы вычисляем сумму дисконтированных вознаграждений, **начиная с временного интервала t**.

- почему? Поскольку наша политика должна только **подкреплять действия на основе последствий**: таким образом, награды, полученные до совершения действия, бесполезны (поскольку они были получены не из-за действия), **важны только те, которые приходят после действия**.

- Прежде чем кодировать это, вы должны прочитать этот раздел [ "не позволяйте прошлому отвлекать вас"](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html#don-t-let-the-past-distract-you) это объясняет, почему мы используем градиент политики "вознаграждение за переход".

Мы используем интересную технику, закодированную [Chris1nexus](https://github.com/Chris1nexus) чтобы **эффективно вычислять доходность на каждом временном шаге**. В комментариях объяснялась процедура. Не стесняйтесь также [проверить объяснение PR](https://github.com/huggingface/deep-rl-class/pull/95)
Но в целом идея состоит в том, чтобы **эффективно вычислять доходность на каждом временном шаге**.

Второй вопрос, который вы можете задать, звучит так: ** почему мы минимизируем потери **? Вы говорили о градиентном подъеме, а не о градиентном спуске?

- Мы хотим максимизировать нашу функцию полезности $J(\theta)$ но в PyTorch, как и в Tensorflow, лучше **минимизировать целевую функцию.**
    - Итак, допустим, мы хотим усилить действие 3 на определенном временном шаге. Перед тренировкой этого действия P равно 0.25.
    - Итак, мы хотим изменить $\theta$ таким образом, чтобы $\pi_\theta(a_3|s; \theta) > 0.25$
    - Поскольку все P должны суммироваться до 1, то функция max $\pi_\theta(a_3|s; \theta)$ **сведет к минимуму вероятность других действий.**
    - Таким образом мы должны в PyTorch **, использовать функцию min $(1 - \pi_\theta(a_3|s; \theta)$).**
    - Эта функция потерь приближается к 0 по мере того, как $\pi_\theta(a_3|s; \theta)$  приближается к 1.
    - Таким образом градиент стремится max $\pi_\theta(a_3|s; \theta)$


In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Помогите нам подсчитать балл во время тренировки
    scores_deque = deque(maxlen=100)
    scores = []
    # Строка 3 псевдокода
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = # ЗАДАЧА: перезагрузите среду
        # Строка 4 псевдокода
        for t in range(max_t):
            action, log_prob = # ЗАДАЧА: получить действие
            saved_log_probs.append(log_prob)
            state, reward, done, _ = # ЗАДАЧА: сделайте шаг в среде env
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        # Строка 6 псевдокода: вычисление возвращаемой награды
        returns = deque(maxlen=max_t) 
        n_steps = len(rewards) 
        # Вычислите дисконтированную доходность на каждом временном шаге
        # как сумму доходности с гамма-дисконтированной доходностью в момент t(G_t) + вознаграждение в момент t

        # За O(N) время, где N - количество временных шагов
        # (это определение дисконтированного дохода G_t следует определению этого количества
        # показано на странице 44 2-го черновика Sutton&Barto 2017)
        # G_t = r_(t+1) + r_(t+2) + ...

        # Учитывая эту формулировку, возвраты на каждом временном шаге t могут быть вычислены
        # путем повторного использования вычисленных будущих возвратов G_(t + 1) для вычисления текущего возврата G_t
        # G_t = r_(t+1) + гамма*G_(t+1)
        # G_(t-1) = r_t + гамма * G_t
        # (это соответствует подходу динамического программирования, с помощью которого мы запоминаем решения, чтобы
        # избежать их многократного вычисления)

        # Это правильно, поскольку приведенное выше эквивалентно (см. также стр. 46 2-го черновика Sutton&Barto 2017)
        # G_(t-1) = r_t + гамма*r_(t+1) + гамма*гамма*r_(t+2) + ...


        ## Учитывая вышесказанное, мы вычисляем доходность на временном шаге t как:
        # гамма[t] * возврат[t] + награда[t]
        #
        ## Мы вычисляем это, начиная с последнего временного шага и заканчивая первым, по порядку
        ## использовать представленную выше формулу и избежать избыточных вычислений, которые потребовались
        ## бы если бы мы делали это от начала до конца.

        ## Следовательно, очередь "returns" будет содержать возвраты в хронологическом порядке, от t=0 до t=n_steps
        ## благодаря функции appendleft(), которая позволяет добавлять к позиции 0 за постоянное время O(1)
        ## обычный список python вместо этого потребовал бы O (N) для этого.
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft(    ) # ЗАДАЧА: завершите здесь       
       
        ## стандартизация результатов используется для того, чтобы сделать обучение более стабильным
        eps = np.finfo(np.float32).eps.item()
        
        ## eps - это наименьшее представимое значение с плавающей точкой, которое
        # добавляется к стандартному отклонению доходности, чтобы избежать числовой нестабильности
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)
        
        # Строка 7:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()
        
        # Строка 8: PyTorch предпочитает градиентный спуск 
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
    return scores

#### Решение

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []
    # Line 3 of pseudocode
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        # Line 4 of pseudocode
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        # Line 6 of pseudocode: calculate the return
        returns = deque(maxlen=max_t) 
        n_steps = len(rewards) 
        # Compute the discounted returns at each timestep,
        # as 
        #      the sum of the gamma-discounted return at time t (G_t) + the reward at time t
        #
        # In O(N) time, where N is the number of time steps
        # (this definition of the discounted return G_t follows the definition of this quantity 
        # shown at page 44 of Sutton&Barto 2017 2nd draft)
        # G_t = r_(t+1) + r_(t+2) + ...
        
        # Given this formulation, the returns at each timestep t can be computed 
        # by re-using the computed future returns G_(t+1) to compute the current return G_t
        # G_t = r_(t+1) + gamma*G_(t+1)
        # G_(t-1) = r_t + gamma* G_t
        # (this follows a dynamic programming approach, with which we memorize solutions in order 
        # to avoid computing them multiple times)
        
        # This is correct since the above is equivalent to (see also page 46 of Sutton&Barto 2017 2nd draft)
        # G_(t-1) = r_t + gamma*r_(t+1) + gamma*gamma*r_(t+2) + ...
        
        
        ## Given the above, we calculate the returns at timestep t as: 
        #               gamma[t] * return[t] + reward[t]
        #
        ## We compute this starting from the last timestep to the first, in order
        ## to employ the formula presented above and avoid redundant computations that would be needed 
        ## if we were to do it from first to last.
        
        ## Hence, the queue "returns" will hold the returns in chronological order, from t=0 to t=n_steps
        ## thanks to the appendleft() function which allows to append to the position 0 in constant time O(1)
        ## a normal python list would instead require O(N) to do this.
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft( gamma*disc_return_t + rewards[t]   )    
            
        ## standardization of the returns is employed to make training more stable
        eps = np.finfo(np.float32).eps.item()
        ## eps is the smallest representable float, which is 
        # added to the standard deviation of the returns to avoid numerical instabilities        
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)
        
        # Line 7:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()
        
        # Line 8: PyTorch prefers gradient descent 
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
    return scores

## Тренируй это
- Теперь мы готовы обучать нашего агента.
- Но сначала мы определяем переменную, содержащую все обучающие гиперпараметры.
- Вы можете изменить параметры тренировки (и должны 😉)

In [ ]:
cartpole_hyperparameters = {
    "h_size": 16,
    "n_training_episodes": 1000,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

In [ ]:
# Создайте политику и разместите ее на устройстве
cartpole_policy = Policy(cartpole_hyperparameters["state_space"], cartpole_hyperparameters["action_space"], cartpole_hyperparameters["h_size"]).to(device)
cartpole_optimizer = optim.Adam(cartpole_policy.parameters(), lr=cartpole_hyperparameters["lr"])

In [ ]:
scores = reinforce(cartpole_policy,
                   cartpole_optimizer,
                   cartpole_hyperparameters["n_training_episodes"], 
                   cartpole_hyperparameters["max_t"],
                   cartpole_hyperparameters["gamma"], 
                   100)

## Определите метод оценки 📝
- Здесь мы определяем метод оценки, который мы собираемся использовать для тестирования нашего Reinforce агента.

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
  """
  Оцените агента на количестве эпизодов `n_eval_episodes` и верните среднее вознаграждение и std вознаграждение.
  :param env: Среда оценки
  :param n_eval_episodes: Количество эпизодов для оценки агента
  :param policy: Reinforce агент
  """
  episode_rewards = []
  for episode in range(n_eval_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0
    
    for step in range(max_steps):
      action, _ = policy.act(state)
      new_state, reward, done, info = env.step(action)
      total_rewards_ep += reward
        
      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## Оцените нашего агента 📈

In [ ]:
evaluate_agent(eval_env, 
               cartpole_hyperparameters["max_t"], 
               cartpole_hyperparameters["n_evaluation_episodes"],
               cartpole_policy)

### Опубликуйте нашу обученную модель на хабе 🔥
Теперь, когда мы увидели, что получили хорошие результаты после обучения, мы можем опубликовать нашу обученную модель на хабе 🤗 с помощью одной строки кода.

Вот пример карточки модели:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/modelcard.png"/>